In [ ]:
import os
import glob
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.windows import from_bounds
import numpy as np
from tqdm import tqdm
import pandas as pd

In [24]:
LULC_SOURCE_FILE = './source/LU-CH/LU-CH_2018all.tif'
LULC_OUTPUT_PATH = './dataset/lulc'

SWISSIMAGE_PATH = './dataset/base'
VARIABLE_FILE = './source/be-b-00.03-37-area-var-v01.xlsx'

SAVE_AS_AS17 = True

In [22]:
mapping_to_as27 = {
    0: [0],
    1: [1, 2],
    2: [3, 4, 5, 6, 7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16, 17, 18,19],
    7: [20, 21], 
    8: [22, 23],
    9: [24, 25, 26, 27, 28, 29, 30],
    10: [ 31, 32, 33, 34, 35, 36],
    11: [ 37, 38],
    12: [ 39],
    13: [ 40],
    14: [ 41],
    15: [ 42],
    16: [ 43, 44],
    17: [ 45],
    18: [ 46, 47, 48, 49],
    19: [ 50, 51],
    20: [ 52, 53, 54, 55, 56],
    21: [ 57],
    22: [ 58, 59, 60], 
    23: [ 61],
    24: [ 62, 63],
    25: [ 64, 65, 66, 67, 68],
    26: [ 69, 70, 71],
    27: [ 72]
}

mapping_to_as17 = {
    0: [0],
    1:  [1],
    2:  [2, 3, 4, 5, 6],
    3:  [6, 7, 8],
    4:  [9],
    5:  [10],
    6:  [11, 12, 13],
    7:  [14],
    8:  [15, 16],
    9:  [17, 18],
    10: [19, 20],
    11: [21],
    12: [22],
    13: [23],
    14: [24],
    15: [25],
    16: [26],
    17: [27]
}

mapping_to_forest_only = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17],
    1: [10]
}

m_as72_to_as27 = [None] * 73
m_as27_to_as17 = [None] * 28
m_as17_to_forest = [None] * 18

for key, map in mapping_to_as27.items():
    for child in map:
        m_as72_to_as27[child] = key

for key, map in mapping_to_as17.items():
    for child in map:
        m_as27_to_as17[child] = key

for key, map in mapping_to_forest_only.items():
    for child in map:
        m_as17_to_forest[child] = key


m_as72_to_as27 = np.array(m_as72_to_as27)
m_as27_to_as17 = np.array(m_as27_to_as17)
m_as17_to_forest = np.array(m_as17_to_forest)

In [ ]:
variables = pd.read_excel(VARIABLE_FILE, sheet_name='Codes AS_72')
variables['CODE_AS27'] = variables['CODE'].apply(lambda c: m_as72_to_as27[c])
variables['CODE_AS17'] = variables['CODE_AS27'].apply(lambda c: m_as27_to_as17[c])
variables['CODE_FOREST'] = variables['CODE_AS17'].apply(lambda c: m_as17_to_forest[c])
variables


In [18]:
def read_base_map_tile(path):
    with rasterio.open(path) as dataset:
        tile_bounds = dataset.bounds

    return tile_bounds

def read_lulc_tile(tile_bounds):
    with rasterio.open(LULC_SOURCE_FILE) as dataset:
        window = from_bounds(tile_bounds.left, tile_bounds.bottom, tile_bounds.right, tile_bounds.top, transform=dataset.transform)
        tile = dataset.read(1, window=window)

    return tile

def map_lulc_tile_to_as17(tile):
    mapping = m_as72_to_as27
    mapping_2 = m_as27_to_as17

    return mapping_2[mapping[tile]]


In [ ]:
image_files = glob.glob(f"{SWISSIMAGE_PATH}/**.tif")
for base_map_tile_path in tqdm(image_files):
    base_map_info = base_map_tile_path.split('_')
    coords = base_map_info[2].split('-')
    east, north = coords

    bounds = read_base_map_tile(base_map_tile_path)
    lulc_tile = read_lulc_tile(bounds)
    if SAVE_AS_AS17 == True:
        lulc_tile = map_lulc_tile_to_as17(lulc_tile)

    filename = os.path.join(f'{LULC_OUTPUT_PATH}/{east}_{north}.npy')
    if os.path.isfile(filename):
        continue

    np.save(filename, lulc_tile)